In [9]:
import pandas as pd
import os.path as osp
from textattack.shared.utils import words_from_text

2022-07-06 10:27:26.204483: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [7]:
result_pth = './results_a2t_sst2'

In [2]:
log_df = pd.read_csv('../results/log.csv')
out_df = log_df[['original_text', 'perturbed_text', 'result_type']]
out_df

,original_text,perturbed_text,result_type
0,it 's a charming and often affecting journey .,it 's a charming and normally affecting journe...,Failed
1,unflinchingly bleak and desperate,unflinchingly grim and desperate,Failed
2,allows us to hope that nolan is poised to emba...,authorizes us to hopes that nolan is prepped t...,Failed
3,"the acting , costumes , music , cinematography...","the acting , costumes , music , cinematography...",Successful
4,"it 's slow -- very , very slow .","it 's slow -- very , very slower .",Failed
5,although laced with humor and a few fanciful t...,although laced with mood and a few fanciful to...,Failed
6,a sometimes tedious film .,a occasionally tedious movies .,Failed
7,or doing last year 's taxes with your ex-wife .,or doing last year 's taxation with your ex-wi...,Failed
8,you do n't have to know about music to appreci...,you do n't have to know about music to thankfu...,Failed
9,"in exactly 89 minutes , most of which passed a...","in exactly 89 minutes , most of which passed a...",Failed


In [3]:
transformation_log = pd.read_csv('../results/transformation.csv', index_col=0, names = ["transformation_id","transformation",
            "prev_text", "after_text", "prev_target", "after_target","from_modified_indices", "to_modified_indices", "changes"])
transformation_log

,transformation,prev_text,after_text,prev_target,after_target,from_modified_indices,to_modified_indices,changes
transformation_id,,,,,,,,
0,"{'class': 'WordSwapEmbedding', 'max_candidates...",0,1,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
1,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,3,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
2,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,4,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
3,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,5,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
4,"{'class': 'WordSwapEmbedding', 'max_candidates...",2,6,-1,-1,{3},{3},"['replace: [3,4]-[3,4]']"
...,...,...,...,...,...,...,...,...
2075,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2095,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"
2076,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2096,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"
2077,"{'class': 'WordSwapEmbedding', 'max_candidates...",1982,2097,-1,-1,{15},{15},"['replace: [15,16]-[15,16]']"


In [4]:
edge_to_transformation = transformation_log.set_index(['prev_text', 'after_text'])
edge_to_transformation

transformation  \
prev_text after_text                                                      
0         1           {'class': 'WordSwapEmbedding', 'max_candidates...   
2         3           {'class': 'WordSwapEmbedding', 'max_candidates...   
          4           {'class': 'WordSwapEmbedding', 'max_candidates...   
          5           {'class': 'WordSwapEmbedding', 'max_candidates...   
          6           {'class': 'WordSwapEmbedding', 'max_candidates...   
...                                                                 ...   
1982      2095        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2096        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2097        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2098        {'class': 'WordSwapEmbedding', 'max_candidates...   
          2099        {'class': 'WordSwapEmbedding', 'max_candidates...   

                      prev_target  after_target from_modified_indices  \
prev_text after_text                                                    
0         1                    -1            -1                   {3}   
2         3                    -1            -1                   {3}   
          4                    -1            -1                   {3}   
          5                    -1            -1                   {3}   
          6                    -1            -1                   {3}   
...                           ...           ...                   ...   
1982      2095                 -1            -1                  {15}   
          2096                 -1            -1                  {15}   
          2097                 -1            -1                  {15}   
          2098                 -1            -1                  {15}   
          2099                 -1            -1                  {15}   

                     to_modified_indices                       changes  
prev_text after_text                                                    
0         1                          {3}      ['replace: [3,4]-[3,4]']  
2         3                          {3}      ['replace: [3,4]-[3,4]']  
          4                          {3}      ['replace: [3,4]-[3,4]']  
          5                          {3}      ['replace: [3,4]-[3,4]']  
          6                          {3}      ['replace: [3,4]-[3,4]']  
...                                  ...                           ...  
1982      2095                      {15}  ['replace: [15,16]-[15,16]']  
          2096                      {15}  ['replace: [15,16]-[15,16]']  
          2097                      {15}  ['replace: [15,16]-[15,16]']  
          2098                      {15}  ['replace: [15,16]-[15,16]']  
          2099                      {15}  ['replace: [15,16]-[15,16]']  

[2080 rows x 6 columns]

In [5]:
edges_df = transformation_log[['prev_text','after_text']]
forward_edges_df = edges_df.set_index('prev_text')
backward_edges_df = edges_df.set_index('after_text')

In [6]:
id_to_text = pd.read_csv('../results/text.csv', index_col="text_id", names = ["text_id", "text"])
text_to_id = pd.read_csv('../results/text.csv', index_col="text", names = ["text_id", "text"])

## Transformation History

In [1]:
from lineage import InferQuery
queryAPI = InferQuery(dir_pth='./results_a2t_sst2')

In [3]:
api_trace = queryAPI.get_trace_of_output_idx(25)
api_trace

[(2504,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formula romantic comedy . "),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{21}',
  '{21}'),
 [('replace', (21, 22), (21, 22))],
 (2505,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from stale parody to raunchy sex gags to formulas romantic comedy . "),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{14}',
  '{14}'),
 [('replace', (14, 15), (14, 15))],
 (2529,
  "nicks , seemingly uncertain what 's going to make people laugh , runs the gamut from archaic parody to raunchy sex gags to formulas romantic comedy . "),
 ("{'class': 'WordSwapEmbedding', 'max_candidates': 20, 'embedding': WordEmbedding}",
  '{12}',
  '{12}'),
 [('replace', (12, 13), (12, 13))],
 (2545,
  "nicks , seemingly uncertain what 's going to make people laugh , runs 

In [5]:
all_traces = queryAPI.get_traces_of_all_outputs()

In [ ]:
all_edits = {}
for record in all_traces:
    if isinstance(record, list):
        for edit in record:
            op = edit[0]
            from_text = 
            all_edits[op] = words_from_text()

In [6]:
len(all_traces)

806